In [ ]:
pip install fuzzywuzzy

In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
movies='/content/movies.csv'
ratings='/content/ratings.csv'


In [ ]:
df_movies=pd.read_csv(movies, usecols=['movieId','title'], dtype={'movieId':'int32','title':'str'})
df_ratings=pd.read_csv(ratings, usecols=['userId','movieId','rating'],dtype={'userId':'int32','movieId':'int32','rating':'float32'})


In [ ]:
df_movies.head(10)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [ ]:
df_ratings.head(10)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
5,1,70,3.0
6,1,101,5.0
7,1,110,4.0
8,1,151,5.0
9,1,157,5.0


In [ ]:
movies_users=df_ratings.pivot(index='movieId', columns='userId',values='rating').fillna(0)
mat_movies_users=csr_matrix(movies_users.values)


In [ ]:
model_knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)
model_knn.fit(mat_movies_users)


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [ ]:
def recommender(movie_name, data,model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df_movies['title'])[2]
    print('Movie Selected: ',df_movies['title'][idx], 'Index: ',idx)
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies['title'][i].where(i!=idx))

recommender('iron man', mat_movies_users, model_knn,10)


Movie Selected:  Iron Man (2008) Index:  6743
Searching for recommendations.....
6302                                 Gridiron Gang (2006)
7322         Human Centipede, The (First Sequence) (2009)
6268                                   Half Nelson (2006)
6882                                     Blindness (2008)
6425                                          Next (2007)
6653                    Drained (O cheiro do Ralo) (2006)
6909    Boy in the Striped Pajamas, The (Boy in the St...
6339        Santa Clause 3: The Escape Clause, The (2006)
6718                        Assembly (Ji jie hao) (2007) 
6092                        Fun with Dick and Jane (2005)
Name: title, dtype: object
